In [1]:
!pip install spyprot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.0 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.6 MB/s eta 0:00:00
  Created wheel for pysolr: filename=pysolr-3.9.0-py2.py3-non

In [1]:
import spyprot
import pandas as pd
from tqdm import tqdm

/home/jovyan/my-conda-envs/tmp/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


### Test Uniprot search

In [10]:
uni = spyprot.UniprotSearch(
    ['xref_interpro', 'xref_pfam'], accessions=['A0A011VZ55'])
res = uni.get()
print(res['A0A011VZ55'])

['IPR029028;IPR003742;IPR029026;', 'PF02590;']


## Add IPRO and PFAM to the csv

In [3]:
# size of accessions list in the UniprotSearch, 100 looks like reasonable constant, bigger numbers do weird things
SIZE = 100

FILE_IN_NAME = "spout_all_unknotted.csv"
FILE_OUT_NAME = "spout_all_unknotted_ipro.csv"

In [4]:
tqdm.pandas()
df = pd.read_csv(FILE_IN_NAME)
df['ID'] = df['ID'].apply(lambda x: x.split(sep='-')[1])

print(df.head())

accessions = df['ID'].tolist()
print(accessions[:10])

final_res = {}

for i in tqdm(range(0, len(accessions) - SIZE, SIZE)):
    uni = spyprot.UniprotSearch(['xref_interpro', 'xref_pfam'], accessions=accessions[i:i + SIZE])
    res = uni.get()
    final_res.update(res)
    
uni = spyprot.UniprotSearch(['xref_interpro''xref_pfam'], accessions=accessions[len(accessions) - SIZE:])
res = uni.get()
final_res.update(res)


df['InterPro'] = df['ID'].progress_apply(lambda x: final_res[x][0])
df['Domain architecture'] = df['ID'].progress_apply(lambda x: final_res[x][1])


           ID  latestVersion  globalMetricValue  uniprotStart  uniprotEnd  \
0  A0A009NLX4              4              94.56             1         306   
1  A0A010QHM8              4              74.75             1         575   
2  A0A010R445              4              91.31             1         353   
3  A0A014C010              4              94.69             1         306   
4  A0A014CD17              4              94.38             1         306   

                                     uniprotSequence  Length  
0  MALRHFLTLRDLSTLELNRILERASELKKMQHDNKVYQPFVGKVLG...     306  
1  MSQLRDEKDNTTLNDSKESTNPKVVVDSVFDTSEKLFLGGIDDGSD...     575  
2  MKQTAFRVTRQALNGAQSRAYSQSTGPRHLMSIADLSPAEFATLVR...     353  
3  MALRHFLTLRDLSTLELNRVLQRASELKKMQQSNKVYQPFVGKVLG...     306  
4  MALRHFLTLRDLSTLELNRVLQRASELKKMQQSNKVYQPFVGKVLG...     306  
['A0A009NLX4', 'A0A010QHM8', 'A0A010R445', 'A0A014C010', 'A0A014CD17', 'A0A015X2G2', 'A0A015YLP7', 'A0A016URG5', 'A0A016WIA3', 'A0A017TC45']


100%|██████████| 331052/331052 [00:00<00:00, 1066553.13it/s]


In [6]:
print(df.head())

           ID  latestVersion  globalMetricValue  uniprotStart  uniprotEnd  \
0  A0A009NLX4              4              94.56             1         306   
1  A0A010QHM8              4              74.75             1         575   
2  A0A010R445              4              91.31             1         353   
3  A0A014C010              4              94.69             1         306   
4  A0A014CD17              4              94.38             1         306   

                                     uniprotSequence  Length  \
0  MALRHFLTLRDLSTLELNRILERASELKKMQHDNKVYQPFVGKVLG...     306   
1  MSQLRDEKDNTTLNDSKESTNPKVVVDSVFDTSEKLFLGGIDDGSD...     575   
2  MKQTAFRVTRQALNGAQSRAYSQSTGPRHLMSIADLSPAEFATLVR...     353   
3  MALRHFLTLRDLSTLELNRVLQRASELKKMQQSNKVYQPFVGKVLG...     306   
4  MALRHFLTLRDLSTLELNRVLQRASELKKMQQSNKVYQPFVGKVLG...     306   

                                            InterPro  
0  IPR006132;IPR006130;IPR036901;IPR006131;IPR002...  
1                               IPR004648;

In [7]:
df.to_csv(FILE_OUT_NAME, index=False)